In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import re

In [3]:
from copy import deepcopy
from collections import deque

class Tile:
    def __init__(self,tile_number,grid):
        self.number = tile_number
        self.grid = grid
        self.sides = self._get_sides(grid)
        self.matched = deque([ None for _ in range(4)])
        self.unmatched = [i for i in range(4)]
    
    def _get_sides(self,grid):
        top=grid[0]
        bottom=grid[-1]
        left = ''.join([ r[0] for r in grid])
        right= ''.join([ r[-1] for r in grid])
        
        return [top, right, bottom, left]
    
    def get_unmatched(self):
        indices = self.unmatched.copy()
        edges =  [ self.sides[e] for e in indices]
        return indices, edges
    
    def update_match(self,side_index, other_tile):
        self.matched[side_index] = other_tile
        self.unmatched.remove(side_index)
        
    def get_matching_edge(self, other_tile):
        for i, tile in enumerate(self.matched):
            if tile == other_tile:
                return self.sides[i]
        return None
        
    def align_image(self,other_edge,other_tile, edge_idx):
        matching_grid = [2,3,0,1]
        matched = self.matched.copy()
        
        this_edge = ''    
        for it, otile in enumerate(matched):
            if otile == other_tile:
                this_edge = self.sides[it]
        match_idx = matching_grid[edge_idx]
        
        grid = self.grid.copy()
        cgrid = [ [r for r in row ] for row in grid]
        
        new_grid = grid.copy()
        
        while matched[match_idx] !=  other_tile:
            new_grid = [''.join([ cgrid[9-j][i] for j in range(10)]) for i in range(10)]
            cgrid = [[r for r in row ] for row in new_grid]
            last = matched.pop()
            matched.appendleft(last)
        
        if sum([x==y for x,y in zip(this_edge, other_edge)]) ==10:
            edges = self._get_sides(new_grid)
            self.grid = new_grid
            self.sides = edges
            self.matched = matched
        
            return
        
        if sum([x==y for x,y in zip(reversed(this_edge), other_edge)]) ==10:
            if match_idx%2 == 0:
                new_grid = self.flip_y_axis(new_grid.copy())
                tmp= deepcopy(matched[1])
                matched[1]=matched[3]
                matched[3]=tmp
                
            else:
                new_grid = self.flip_x_axis(new_grid.copy())
                tmp= deepcopy(matched[0])
                matched[0]=matched[2]
                matched[2]=tmp
            
            edges = self._get_sides(new_grid)
            self.grid = new_grid
            self.sides = edges
            self.matched = matched
            return
        
        
    def __str__(self):
        return '\n'.join(self.grid)
    
        
    def flip_x_axis(self,grid):
        new_grid = [ grid[i] for i in range(9,-1,-1)]
        return new_grid
        
    def flip_y_axis(self,grid):
        new_grid = [ row[::-1] for row in grid]
        return new_grid 
        
        

In [4]:
tiles = {}

with open('data/input_day20.txt', 'r') as f:
#with open('test_20.txt', 'r') as f:
    line = True
    while line:
        line = f.readline()
        m=re.match('Tile (\d+):', line)
        number = int(m.groups()[0])
        grid=[ f.readline().strip() for _ in range(10)]
        
        tiles[number]=(Tile(number, grid))
            
        line = f.readline()
        
        

In [5]:
def get_corner_tiles(tile_list):
    for i,orig_tile in enumerate(tile_list[:-1]):
        orig_idx, orig_edges = orig_tile.get_unmatched()
        for other_tile in tile_list[i+1:]:
            if len(orig_idx) ==0:
                break
            other_idx, other_edges = other_tile.get_unmatched()
            if len(other_idx) ==0:
                continue
            
            matched = False
            for ref_i in range(len(orig_idx)):
                for other_i in range(len(other_idx)):
                    matching = sum([x ==y for x,y in zip(orig_edges[ref_i], other_edges[other_i])])
                    if matching == 10:
                        o_idx = other_idx.pop(other_i)
                        o_ed = other_edges.pop(other_i)
                        r_idx = orig_idx.pop(ref_i)
                        r_ed = orig_edges.pop(ref_i)
                        
                        other_tile.update_match(o_idx, orig_tile.number)
                        orig_tile.update_match(r_idx, other_tile.number)
                        matched = True
                        break
                        
                    reverse_matching = sum([x==y for x,y in zip(orig_edges[ref_i], reversed(other_edges[other_i]))])
                    if reverse_matching == 10:
                        o_idx = other_idx.pop(other_i)
                        o_ed = other_edges.pop(other_i)
                        r_idx = orig_idx.pop(ref_i)
                        r_ed = orig_edges.pop(ref_i)
                        
                        other_tile.update_match(o_idx, orig_tile.number)
                        orig_tile.update_match(r_idx, other_tile.number)
                        matched = True
                        break
                if matched:
                    break

def get_part_one(tile_list):
    get_corner_tiles(tile_list)
    
    corners = [ t.number for t in tiles.values() if len(t.unmatched) == 2]
    t=1
    for c in corners:
        t*=c
    return t

In [6]:
get_part_one(list(tiles.values()))

17148689442341

In [7]:
corners = [ t.number for t in tiles.values() if len(t.unmatched) == 2]

In [8]:
corners

[3181, 2543, 1453, 1459]

In [9]:
[ (t,tiles[t].unmatched) for t in corners]

[(3181, [2, 3]), (2543, [2, 3]), (1453, [0, 3]), (1459, [1, 2])]

In [10]:
def running_row(starting_number, tiles_dict):
    
    idx = starting_number 
    cnt=0
    
    whole_grid=[]
    while tiles_dict[idx].matched[2] is not None :
        row=[tiles_dict[idx]]
        while tiles_dict[idx].matched[1] is not None :
            next_index = tiles_dict[idx].matched[1]
            
            this_side=tiles_dict[idx].sides[1]
            
            tiles_dict[next_index].align_image(tiles_dict[idx].sides[1],idx, 1)
            row.append(tiles_dict[next_index])
            idx = next_index
        whole_grid.append(row)
        
        below_idx = row[0].matched[2]
        tiles_dict[below_idx].align_image(row[0].sides[2],row[0].number,2)
        idx=below_idx
    row=[tiles_dict[idx]]
    while tiles_dict[idx].matched[1] is not None :
        next_index = tiles_dict[idx].matched[1]
        
        this_side=tiles_dict[idx].sides[1]
        
        tiles_dict[next_index].align_image(tiles_dict[idx].sides[1],idx, 1)
        row.append(tiles_dict[next_index])
        idx = next_index
    whole_grid.append(row)
    
    return whole_grid

def aligning_jigsaw(tiles_dict, starting_idx):
    row = []

In [11]:
for _ in range(5):
    test =running_row(1453, tiles)

In [12]:
row = test[0]
with open('tmp.txt','w') as f:
    for row in test:
        for i in range(10):
            print('\t'.join([ t.grid[i] for t in row]),file=f)
        print('\n',file=f)
    

In [13]:
jigsaw=[]
for row in test:
    cur_row = [ ''.join([ t.grid[r][1:-1] for t in row]) for r in range(1,9)]
    jigsaw.extend(cur_row)

In [14]:
len(test)
len(test[0])

12

12

In [15]:
len(jigsaw)
len(jigsaw[0])

96

96

In [16]:
def check_for_sea_monsters(grid):
    
    def check_cur_spot(new_grid, loc):
        if sum([ (new_grid[loc[0]+i][loc[1]+j] == '#' 
                  or new_grid[loc[0]+i][loc[1]+j] == 'O')  
                for i,j in monster_indices ]) == len(monster_indices):
            for x,y in monster_indices :
                new_grid[loc[0]+x][loc[1]+y] = 'O'
        return new_grid
    
    check_grid = deepcopy(grid)
    check_grid = [[r for r in row ] for row in check_grid]
    sea_monster = [
    '                  # ',
    '#    ##    ##    ###',
    ' #  #  #  #  #  #   ',
    ]

    monster_indices = []
    for i,row in enumerate(sea_monster) :
        for j,symbol in enumerate(row)  :
            if symbol=='#':
                monster_indices.append((i,j))
    filter_size = (len(sea_monster),len(sea_monster[0]))
    
    
    max_row = len(grid) - filter_size[0]+1
    max_col = len(grid[0]) - filter_size[1]+1
    
    for r in range(max_row):
        for c in range(max_col):
            check_grid = check_cur_spot(check_grid, (r,c))
            
    return [''.join(row) for row in check_grid ]
        
    

In [17]:
def flip_x_axis(grid):
    row_count = len(grid)
    new_grid = [ grid[i] for i in range(row_count-1,-1,-1)]
    return new_grid
    
def flip_y_axis(grid):
    new_grid = [ row[::-1] for row in grid]
    return new_grid 

def part_two_monsters(jig_saw):
    jgrid = [[r for r in row ] for row in jig_saw]
    
    for _ in range(4):
        j_rows = len(jgrid)
        j_cols = len(jgrid[0])
        
        new_grid = [''.join([ jgrid[j_rows-1-j][i] for j in range(j_rows)]) for i in range(j_cols)]
        jgrid = [[r for r in row ] for row in new_grid]
        
        marked = check_for_sea_monsters(new_grid)

        if ''.join(marked).count('O') != 0:
            return ''.join(marked).count('#'), marked
     
        flipped_grid = flip_x_axis(new_grid)
        marked = check_for_sea_monsters(flipped_grid)
        if ''.join(marked).count('O') != 0:
            return ''.join(marked).count('#'), marked
        
        flipped_grid = flip_y_axis(new_grid)
        marked = check_for_sea_monsters(flipped_grid)
        if ''.join(marked).count('O') != 0:
            return ''.join(marked).count('#'), marked

In [18]:
count, marked = part_two_monsters(jigsaw)

In [19]:
count

2009

In [20]:
temp = [ r.replace('.', ' ') for r in marked]

In [21]:
temp

['  #  #   ##  # #      ## #    ##   #####    # # ### #     ##          # #         #     #       ',
 '          ####    #          #   # #            #  #   ##   ###     ##          # ##          # ',
 '#    #   # #    #  #     ##     #              #      O # #     #  ###  ###      #       O#   # ',
 ' #     #       ###      #  #   #   #O  # OO##  OO##  OOO   ###  # #   #O    OO   #OO ## OOO     ',
 ' #     #    # # ##      O #  ## # #  O #O  O  O# O  O        #   # #  # O #O  O  O #O  O        ',
 '#     O # #OO#   OO    OOO #    #         #  # #           #     # #      ###            #   ## ',
 ' ## #  O  O# O# O #O  O    # #   ##     #  # #   #             ##  #    #    # # #   #   #      ',
 '###                #      # #    #      ##       #  #            #O   #  #   #   # #        #   ',
 '        #              # #   #    #  # #   ## # O# # OO    OO    OOO #  #  # #     #  #         ',
 '## # # ## #           #     #  #    #  #   #  # #O  O #O  O  O  O #    #  ##       ##  # 